# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [72]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score

import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [8]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#data.status_code
data

{'companyName': 'Apple Inc',
 'marketcap': 2263192958208,
 'week52high': 147.46,
 'week52low': 71.16,
 'week52highSplitAdjustOnly': 150.53,
 'week52lowSplitAdjustOnly': 72.74,
 'week52change': 0.9514210354560609,
 'sharesOutstanding': 17172889406,
 'float': 0,
 'avg10Volume': 88579312,
 'avg30Volume': 88845504,
 'day200MovingAvg': 130.72,
 'day50MovingAvg': 132.05,
 'employees': 152059,
 'ttmEPS': 3.83,
 'ttmDividendRate': 0.8302853962151342,
 'dividendYield': 0.006113984185029861,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-22',
 'nextEarningsDate': '2021-04-16',
 'peRatio': 36.3213368508093,
 'beta': 1.362909244973013,
 'maxChangePercent': 51.89510627569569,
 'year5ChangePercent': 4.607324455034229,
 'year2ChangePercent': 1.663963701730564,
 'year1ChangePercent': 0.9984458231256862,
 'ytdChangePercent': 0.01415015222859659,
 'month6ChangePercent': 0.1790389716612561,
 'month3ChangePercent': -0.03393233557614548,
 'month1ChangePercent': 0.09947398977852392,
 'day30ChangePerc

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [9]:
data['year1ChangePercent']

0.9984458231256862

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [24]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote,news,chart&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data['AAPL']['stats'])
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index = True
        )
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,139.12,0.850767,N/A
1,AAL,21.68,1.059646,N/A
2,AAP,207.49,0.799367,N/A
3,AAPL,137.30,1.004288,N/A
4,ABBV,115.26,0.441610,N/A
...,...,...,...,...
95,CINF,109.92,0.450260,N/A
96,CL,81.80,0.149937,N/A
97,CLX,196.97,0.014110,N/A
98,CMA,71.39,1.377058,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [30]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,24,ALGN,633.47,2.338429,N/A
1,78,CARR,44.67,1.865300,N/A
2,23,ALB,169.21,1.760879,N/A
3,29,AMAT,141.17,1.736232,N/A
4,34,AMP,250.73,1.479662,N/A
5,25,ALK,70.76,1.476629,N/A
6,45,APTV,145.48,1.454090,N/A
7,90,CFG,45.43,1.404901,N/A
8,98,CMA,71.39,1.377058,N/A
9,16,AES,28.90,1.333096,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [64]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1e7
1e7


In [40]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,24,ALGN,633.47,2.338429,315
1,78,CARR,44.67,1.865300,4477
2,23,ALB,169.21,1.760879,1181
3,29,AMAT,141.17,1.736232,1416
4,34,AMP,250.73,1.479662,797
5,25,ALK,70.76,1.476629,2826
6,45,APTV,145.48,1.454090,1374
7,90,CFG,45.43,1.404901,4402
8,98,CMA,71.39,1.377058,2801
9,16,AES,28.90,1.333096,6920


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [123]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM_Score'
]

hqm_dataframe = pd.DataFrame(columns= hqm_columns)
#hqm_dataframe

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns
            ),
            ignore_index = True
        )

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM_Score
0,A,139.16,N/A,0.833354,N/A,0.295362,N/A,0.0848583,N/A,0.127313,N/A,N/A
1,AAL,21.72,N/A,1.08079,N/A,0.693247,N/A,0.343008,N/A,-0.0523835,N/A,N/A
2,AAP,201.96,N/A,0.775534,N/A,0.301454,N/A,0.229938,N/A,0.118223,N/A,N/A
3,AAPL,138.74,N/A,0.983856,N/A,0.175089,N/A,-0.0332684,N/A,0.0986746,N/A,N/A
4,ABBV,112.46,N/A,0.435294,N/A,0.362654,N/A,0.017749,N/A,0.0782965,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,118.88,N/A,0.420688,N/A,0.173177,N/A,0.105496,N/A,0.0939579,N/A,N/A
501,ZBH,182.75,N/A,0.606593,N/A,0.248677,N/A,0.106827,N/A,0.14343,N/A,N/A
502,ZBRA,507.23,N/A,1.49923,N/A,0.666541,N/A,0.218901,N/A,0.0773015,N/A,N/A
503,ZION,54.98,N/A,0.971683,N/A,0.667398,N/A,0.141994,N/A,0.0128963,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [124]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM_Score
0,A,139.16,N/A,0.833354,0.673267,0.295362,0.538614,0.0848583,0.314851,0.127313,0.859406,N/A
1,AAL,21.72,N/A,1.08079,0.811881,0.693247,0.90495,0.343008,0.948515,-0.0523835,0.0257426,N/A
2,AAP,201.96,N/A,0.775534,0.633663,0.301454,0.552475,0.229938,0.839604,0.118223,0.813861,N/A
3,AAPL,138.74,N/A,0.983856,0.760396,0.175089,0.348515,-0.0332684,0.0594059,0.0986746,0.683168,N/A
4,ABBV,112.46,N/A,0.435294,0.340594,0.362654,0.653465,0.017749,0.138614,0.0782965,0.538614,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,118.88,N/A,0.420688,0.328713,0.173177,0.338614,0.105496,0.392079,0.0939579,0.647525,N/A
501,ZBH,182.75,N/A,0.606593,0.49505,0.248677,0.453465,0.106827,0.39802,0.14343,0.90495,N/A
502,ZBRA,507.23,N/A,1.49923,0.932673,0.666541,0.881188,0.218901,0.809901,0.0773015,0.526733,N/A
503,ZION,54.98,N/A,0.971683,0.754455,0.667398,0.883168,0.141994,0.544554,0.0128963,0.138614,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [125]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM_Score'] = mean(momentum_percentiles)
 
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM_Score
0,A,139.16,N/A,0.833354,0.673267,0.295362,0.538614,0.0848583,0.314851,0.127313,0.859406,0.596535
1,AAL,21.72,N/A,1.08079,0.811881,0.693247,0.90495,0.343008,0.948515,-0.0523835,0.0257426,0.672772
2,AAP,201.96,N/A,0.775534,0.633663,0.301454,0.552475,0.229938,0.839604,0.118223,0.813861,0.709901
3,AAPL,138.74,N/A,0.983856,0.760396,0.175089,0.348515,-0.0332684,0.0594059,0.0986746,0.683168,0.462871
4,ABBV,112.46,N/A,0.435294,0.340594,0.362654,0.653465,0.017749,0.138614,0.0782965,0.538614,0.417822
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,118.88,N/A,0.420688,0.328713,0.173177,0.338614,0.105496,0.392079,0.0939579,0.647525,0.426733
501,ZBH,182.75,N/A,0.606593,0.49505,0.248677,0.453465,0.106827,0.39802,0.14343,0.90495,0.562871
502,ZBRA,507.23,N/A,1.49923,0.932673,0.666541,0.881188,0.218901,0.809901,0.0773015,0.526733,0.787624
503,ZION,54.98,N/A,0.971683,0.754455,0.667398,0.883168,0.141994,0.544554,0.0128963,0.138614,0.580198


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [126]:
hqm_dataframe.sort_values('HQM_Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM_Score
0,LB,69.25,N/A,5.42377,1,0.977707,0.980198,0.499635,0.994059,0.175025,0.960396,0.983663
1,GPS,34.16,N/A,3.93314,0.99802,0.606098,0.859406,0.532002,0.99802,0.23863,0.99604,0.962871
2,HPQ,35.41,N/A,1.45239,0.926733,0.859287,0.966337,0.387061,0.978218,0.164171,0.948515,0.95495
3,TPR,46.46,N/A,2.44299,0.990099,1.21014,0.986139,0.366236,0.968317,0.130575,0.871287,0.95396
4,AMAT,141.00,N/A,1.68605,0.952475,1.27978,0.988119,0.278188,0.912871,0.164162,0.946535,0.95
5,KSU,304.87,N/A,1.39091,0.908911,0.673994,0.889109,0.392996,0.982178,0.226387,0.988119,0.942079
6,IVZ,26.74,N/A,2.88678,0.994059,0.944599,0.976238,0.283593,0.918812,0.126373,0.857426,0.936634
7,MHK,211.52,N/A,1.63728,0.946535,0.985068,0.982178,0.358518,0.958416,0.123493,0.841584,0.932178
8,PWR,101.76,N/A,1.99364,0.974257,0.588139,0.845545,0.265325,0.89505,0.186369,0.970297,0.921287
9,STX,91.30,N/A,0.950035,0.738614,0.820631,0.954455,0.506012,0.99604,0.231178,0.992079,0.920297


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [127]:
portfolio_input()

Enter the size of your portfolio:1e6


In [128]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price']) 

hqm_dataframe

C:\Users\Mike\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM_Score
0,LB,69.25,288,5.42377,1,0.977707,0.980198,0.499635,0.994059,0.175025,0.960396,0.983663
1,GPS,34.16,585,3.93314,0.99802,0.606098,0.859406,0.532002,0.99802,0.23863,0.99604,0.962871
2,HPQ,35.41,564,1.45239,0.926733,0.859287,0.966337,0.387061,0.978218,0.164171,0.948515,0.95495
3,TPR,46.46,430,2.44299,0.990099,1.21014,0.986139,0.366236,0.968317,0.130575,0.871287,0.95396
4,AMAT,141.00,141,1.68605,0.952475,1.27978,0.988119,0.278188,0.912871,0.164162,0.946535,0.95
5,KSU,304.87,65,1.39091,0.908911,0.673994,0.889109,0.392996,0.982178,0.226387,0.988119,0.942079
6,IVZ,26.74,747,2.88678,0.994059,0.944599,0.976238,0.283593,0.918812,0.126373,0.857426,0.936634
7,MHK,211.52,94,1.63728,0.946535,0.985068,0.982178,0.358518,0.958416,0.123493,0.841584,0.932178
8,PWR,101.76,196,1.99364,0.974257,0.588139,0.845545,0.265325,0.89505,0.186369,0.970297,0.921287
9,STX,91.30,219,0.950035,0.738614,0.820631,0.954455,0.506012,0.99604,0.231178,0.992079,0.920297


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [129]:
writer = pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [130]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [131]:
column_formats={
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template ],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM_Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column( f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [132]:
writer.save()